In [1]:
import sys
print(sys.executable)


/Users/kamayanirai/anaconda3/envs/hfenv/bin/python


In [ ]:
!pip install "transformers[torch]"

zsh:1: 0.26.0 not found


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [1]:
import transformers
print(transformers.__file__)
print(transformers.__version__)


/Users/kamayanirai/anaconda3/envs/hfenv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


/Users/kamayanirai/anaconda3/envs/hfenv/lib/python3.10/site-packages/transformers/__init__.py
4.51.3


In [4]:
from transformers import TrainingArguments
print(TrainingArguments.__module__)  # This will show where the class is coming from
print(dir(TrainingArguments))

transformers.training_args
['_VALID_DICT_FIELDS', '__annotations__', '__class__', '__dataclass_fields__', '__dataclass_params__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__match_args__', '__module__', '__ne__', '__new__', '__post_init__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_dict_torch_dtype_to_str', '_n_gpu', '_no_sync_in_gradient_accumulation', '_setup_devices', 'accelerator_config', 'adafactor', 'adam_beta1', 'adam_beta2', 'adam_epsilon', 'auto_find_batch_size', 'average_tokens_across_devices', 'batch_eval_metrics', 'bf16', 'bf16_full_eval', 'data_seed', 'dataloader_drop_last', 'dataloader_num_workers', 'dataloader_persistent_workers', 'dataloader_pin_memory', 'dataloader_prefetch_factor', 'ddp_backend', 'ddp_broadcast_buffers', 'ddp_bucket_cap_mb', 'ddp_find_unuse

In [ ]:
# --- Imports ---
import pandas as pd
import numpy as np
import torch
from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
import json

with open('ingredient.json', 'r') as f:
    data = json.load(f)

df = pd.DataFrame(data)

# Create mapping dictionaries
unique_labels = df["label"].unique().tolist()
label2id = {label: idx for idx, label in enumerate(unique_labels)}
id2label = {idx: label for label, idx in label2id.items()}

# Map labels to integers
df["labels"] = df["label"].map(label2id)

df = df.drop(columns=["label"])   

# Load into Huggingface Dataset -
dataset = Dataset.from_pandas(df)


# Tokenizer and Model 
model_name = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name, model_max_length=128)


def tokenize_function(examples):
    tokenized = tokenizer(examples["ingredient"], padding= True, truncation=True)
    tokenized["labels"] = examples["labels"]  # Already mapped to int
    return tokenized

# Apply tokenizer
tokenized_dataset = dataset.map(tokenize_function, batched=True)



/Users/kamayanirai/anaconda3/envs/hfenv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Map: 100%|██████████| 293/293 [00:00<00:00, 12114.86 examples/s]


In [ ]:
train_test_split = tokenized_dataset.train_test_split(test_size=0.2)
train_dataset = train_test_split["train"]
eval_dataset = train_test_split["test"]


model = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    num_labels=len(unique_labels),
    id2label=id2label,
    label2id=label2id
)

# Metrics 
def compute_metrics(pred):
    preds = np.argmax(pred.predictions, axis=1)
    labels = pred.label_ids
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='weighted')
    acc = accuracy_score(labels, preds)
    return {"accuracy": acc, "f1": f1, "precision": precision, "recall": recall}

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:

# Trainer Arguments 
training_args = TrainingArguments(
    output_dir="ingredient_model",
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate= 5e-5 ,
    per_device_train_batch_size= 8,
    per_device_eval_batch_size= 8,
    num_train_epochs= 15 ,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=10,
    save_total_limit=1,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    greater_is_better=True,
)

# Trainer 
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    compute_metrics=compute_metrics,
    tokenizer=tokenizer,
)

# Train the model 
trainer.train()



/var/folders/pt/dcyg6j9x4bj4f_2gx863y7mw0000gn/T/ipykernel_52379/1570084787.py:20: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
/Users/kamayanirai/anaconda3/envs/hfenv/lib/python3.10/site-packages/torch/utils/data/dataloader.py:683: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,0.191200,2.017516,0.576271,0.569948,0.576059,0.576271
2,0.140500,2.090262,0.610169,0.593305,0.629349,0.610169
3,0.132000,1.931019,0.661017,0.647215,0.652234,0.661017
4,0.109400,2.102179,0.593220,0.598337,0.612125,0.593220
5,0.029900,2.166304,0.627119,0.633575,0.643607,0.627119
6,0.159500,2.253674,0.677966,0.662188,0.692239,0.677966
7,0.007200,2.094801,0.661017,0.677966,0.701695,0.661017
8,0.045200,2.384314,0.644068,0.650282,0.682246,0.644068
9,0.101900,2.308760,0.627119,0.643309,0.685005,0.627119
10,0.002600,2.178714,0.677966,0.690311,0.720339,0.677966


/Users/kamayanirai/anaconda3/envs/hfenv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/kamayanirai/anaconda3/envs/hfenv/lib/python3.10/site-packages/torch/utils/data/dataloader.py:683: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, then device pinned memory won't be used.
  warnings.warn(warn_msg)
/Users/kamayanirai/anaconda3/envs/hfenv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/kamayanirai/anaconda3/envs/hfenv/lib/python3.10/s

TrainOutput(global_step=450, training_loss=0.05201577324006292, metrics={'train_runtime': 575.2503, 'train_samples_per_second': 6.102, 'train_steps_per_second': 0.782, 'total_flos': 27058187683800.0, 'train_loss': 0.05201577324006292, 'epoch': 15.0})

In [ ]:

# Save the fine-tuned model 
trainer.save_model("ingredient_classifier_final")
tokenizer.save_pretrained("ingredient_classifier_final")

('ingredient_classifier_final/tokenizer_config.json',
 'ingredient_classifier_final/special_tokens_map.json',
 'ingredient_classifier_final/vocab.txt',
 'ingredient_classifier_final/added_tokens.json',
 'ingredient_classifier_final/tokenizer.json')